<a href="https://colab.research.google.com/github/waleogundeji/github_share/blob/master/bert_for_text_classification_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import pandas as pd

In [ ]:
tra = pd.read_csv('gdrive/My Drive/indiana_reports.csv')
tra.head()

,uid,MeSH,Problems,image,indication,comparison,findings,impression
0,1,normal,normal,Xray Chest PA and Lateral,Positive TB test,None.,The cardiac silhouette and mediastinum size ar...,Normal chest x-XXXX.
1,2,Cardiomegaly/borderline;Pulmonary Artery/enlarged,Cardiomegaly;Pulmonary Artery,"Chest, 2 views, frontal and lateral",Preop bariatric surgery.,None.,Borderline cardiomegaly. Midline sternotomy XX...,No acute pulmonary findings.
2,3,normal,normal,Xray Chest PA and Lateral,"rib pain after a XXXX, XXXX XXXX steps this XX...",NaN,NaN,"No displaced rib fractures, pneumothorax, or p..."
3,4,"Pulmonary Disease, Chronic Obstructive;Bullous...","Pulmonary Disease, Chronic Obstructive;Bullous...","PA and lateral views of the chest XXXX, XXXX a...",XXXX-year-old XXXX with XXXX.,None available,There are diffuse bilateral interstitial and a...,1. Bullous emphysema and interstitial fibrosis...
4,5,Osteophyte/thoracic vertebrae/multiple/small;T...,Osteophyte;Thickening;Lung,Xray Chest PA and Lateral,Chest and nasal congestion.,NaN,The cardiomediastinal silhouette and pulmonary...,No acute cardiopulmonary abnormality.


In [ ]:
#txt[txt['findings'].isna()]
txt = tra[['Problems', 'findings']]
txt = txt.rename(columns={"Problems":"y", "findings":"text"})
txt.head()

,y,text
0,normal,The cardiac silhouette and mediastinum size ar...
1,Cardiomegaly;Pulmonary Artery,Borderline cardiomegaly. Midline sternotomy XX...
2,normal,NaN
3,"Pulmonary Disease, Chronic Obstructive;Bullous...",There are diffuse bilateral interstitial and a...
4,Osteophyte;Thickening;Lung,The cardiomediastinal silhouette and pulmonary...


In [ ]:
## filter categories   ##'Pulmonary Atelectasis','Implanted Medical Device', 'Foreign Bodies', 'Atherosclerosis', 
txt = txt[txt["y"].isin(['normal','Calcified Granuloma','Calcinosis','Opacity','Cardiomegaly','Spondylosis',
                         'Granulomatous Disease', 'Osteophyte', 'Fractures, Bone'])][["y","text"]]
txt.head()

,y,text
0,normal,The cardiac silhouette and mediastinum size ar...
2,normal,NaN
5,normal,Heart size and mediastinal contour are within ...
7,normal,"The heart, pulmonary XXXX and mediastinum are ..."
9,Calcified Granuloma,The cardiomediastinal silhouette is within nor...


In [ ]:
df = txt[pd.notnull(txt['text'])].reset_index()
df.head()

,index,y,text
0,0,normal,The cardiac silhouette and mediastinum size ar...
1,5,normal,Heart size and mediastinal contour are within ...
2,7,normal,"The heart, pulmonary XXXX and mediastinum are ..."
3,9,Calcified Granuloma,The cardiomediastinal silhouette is within nor...
4,10,normal,Cardiomediastinal silhouette and pulmonary vas...


In [ ]:
df = df[['text', 'y']]
df.head()

,text,y
0,The cardiac silhouette and mediastinum size ar...,normal
1,Heart size and mediastinal contour are within ...,normal
2,"The heart, pulmonary XXXX and mediastinum are ...",normal
3,The cardiomediastinal silhouette is within nor...,Calcified Granuloma
4,Cardiomediastinal silhouette and pulmonary vas...,normal


In [ ]:
num_classes = len(df["y"].value_counts())
print(num_classes)

9


In [ ]:
# map target 'y' to labels
df['labels'] = df['y'].map({'normal': 0,
                            'Calcified Granuloma': 1,
                            'Calcinosis': 3,
                            'Opacity': 3,
                            'Cardiomegaly': 4,
                            'Spondylosis': 5,
                            'Granulomatous Disease': 6, 
                            'Osteophyte': 7, 
                            'Fractures, Bone': 8})

# drop unused column
df = df.drop(["y"], axis=1)

df.head()

,text,labels
0,The cardiac silhouette and mediastinum size ar...,0
1,Heart size and mediastinal contour are within ...,0
2,"The heart, pulmonary XXXX and mediastinum are ...",0
3,The cardiomediastinal silhouette is within nor...,1
4,Cardiomediastinal silhouette and pulmonary vas...,0


In [ ]:
"""
The labels are:
0 - normal
1 - Calcified Granuloma
2 - Calcinosis
3 - Opacity
4 - Cardiomegaly
5 - Spondylosis
6 - Granulomatous Disease': 6, 
7 - Osteophyte
8 - Fractures, Bone
"""
print(df['labels'].value_counts())

0    1197
1      72
3      61
4      24
5      20
6      18
8      16
7      15
Name: labels, dtype: int64


In [ ]:
!pip install transformers

     |████████████████████████████████| 3.8 MB 5.3 MB/s 
     |████████████████████████████████| 6.6 MB 31.1 MB/s 
     |████████████████████████████████| 67 kB 5.7 MB/s 
     |████████████████████████████████| 596 kB 45.2 MB/s 
     |████████████████████████████████| 895 kB 44.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import tensorflow as tf
from transformers import BertTokenizer

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
token = tokenizer.encode_plus(df['text'].iloc[0], max_length= 50, 
                              truncation=True, padding='max_length', 
                              add_special_tokens=True, return_tensors='tf')

token.input_ids

<tf.Tensor: shape=(1, 50), dtype=int32, numpy=
array([[  101,  1109, 17688, 27316,  1105,  2394,  2050,  1394,  1818,
         2060,  1132,  1439,  2999,  6263,   119,  1247,  1110,  1185,
        26600,  5048, 14494,   119,  1247,  1110,  1185, 17811, 20994,
          119,  1247,  1132,  1185,   161,  3190,  3190,  3190,  1104,
          170,   185,  1513, 12602,   174,  3101, 17268,   119,  1247,
         1110,  1185,  2554,  1104,   102]], dtype=int32)>

In [ ]:
X_input_ids = np.zeros((len(df), 50))
X_attn_masks = np.zeros((len(df), 50))

In [ ]:
def generate_training_data(df, ids, masks, tokenizer):
    for i, text in tqdm(enumerate(df['text'])):
        tokenized_text = tokenizer.encode_plus(text, max_length=50, truncation=True, 
                                               padding='max_length', add_special_tokens=True,
                                               return_tensors='tf')
        ids[i, :] = tokenized_text.input_ids
        masks[i, :] = tokenized_text.attention_mask
    return ids, masks

X_input_ids, X_attn_masks = generate_training_data(df, X_input_ids, X_attn_masks, tokenizer)

0it [00:00, ?it/s]

In [ ]:
labels = np.zeros((len(df), 9))
labels.shape

(1423, 9)

In [ ]:
labels[np.arange(len(df)), df['labels'].values] = 1 # one-hot encoded target tensor
labels

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
# creating a data pipeline using tensorflow dataset utility, creates batches of data for easy loading...
dataset = tf.data.Dataset.from_tensor_slices((X_input_ids, X_attn_masks, labels))
dataset.take(1) # one sample data

<TakeDataset element_spec=(TensorSpec(shape=(50,), dtype=tf.float64, name=None), TensorSpec(shape=(50,), dtype=tf.float64, name=None), TensorSpec(shape=(9,), dtype=tf.float64, name=None))>

In [ ]:
def LabelDatasetMapFunction(input_ids, attn_masks, labels):
    return {
        'input_ids': input_ids,
        'attention_mask': attn_masks
    }, labels

In [ ]:
dataset = dataset.map(LabelDatasetMapFunction) # converting to required format for tensorflow dataset
dataset.take(1)

<TakeDataset element_spec=({'input_ids': TensorSpec(shape=(50,), dtype=tf.float64, name=None), 'attention_mask': TensorSpec(shape=(50,), dtype=tf.float64, name=None)}, TensorSpec(shape=(9,), dtype=tf.float64, name=None))>

In [ ]:
dataset = dataset.shuffle(10000).batch(16, drop_remainder=True) # batch size, drop any left out tensor
dataset.take(1)

<TakeDataset element_spec=({'input_ids': TensorSpec(shape=(16, 50), dtype=tf.float64, name=None), 'attention_mask': TensorSpec(shape=(16, 50), dtype=tf.float64, name=None)}, TensorSpec(shape=(16, 9), dtype=tf.float64, name=None))>

In [ ]:
p = 0.8
train_size = int((len(df)//16)*p) # for each 16 batch of data we will have len(df)//16 samples, take 80% of that for train.
train_size

70

In [ ]:
train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size)

In [ ]:
from transformers import TFBertModel

model = TFBertModel.from_pretrained('bert-base-cased') # bert base model with pretrained weights

Downloading:   0%|          | 0.00/502M [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
# defining 2 input layers for input_ids and attn_masks
input_ids = tf.keras.layers.Input(shape=(50,), name='input_ids', dtype='int32')
attn_masks = tf.keras.layers.Input(shape=(50,), name='attention_mask', dtype='int32')

bert_embds = model.bert(input_ids, attention_mask=attn_masks)[1] # 0 -> activation layer (3D), 1 -> pooled output layer (2D)
intermediate_layer = tf.keras.layers.Dense(512, activation='relu', name='intermediate_layer')(bert_embds)
output_layer = tf.keras.layers.Dense(9, activation='softmax', name='output_layer')(intermediate_layer) # softmax -> calcs probs of classes

label_model = tf.keras.Model(inputs=[input_ids, attn_masks], outputs=output_layer)
label_model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 50)]         0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 50)]         0           []                               
                                                                                                  
 bert (TFBertMainLayer)         multiple             108310272   ['input_ids[0][0]',              
                                                                  'attention_mask[0][0]']         
                                                                                                  
 intermediate_layer (Dense)     (None, 512)          393728      ['bert[3][1]']             

In [ ]:
optim = tf.keras.optimizers.Adam(learning_rate=1e-5, decay=1e-6)
loss_func = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.CategoricalAccuracy('accuracy')

In [ ]:
label_model.compile(optimizer=optim, loss=loss_func, metrics=[acc])

In [ ]:
hist = label_model.fit(train_dataset,
                       validation_data=val_dataset, epochs=2)

Epoch 1/2
70/70 [==============================] - 43s 435ms/step - loss: 0.9329 - accuracy: 0.7679 - val_loss: 0.6314 - val_accuracy: 0.8576
Epoch 2/2
70/70 [==============================] - 28s 402ms/step - loss: 0.5783 - accuracy: 0.8536 - val_loss: 0.4070 - val_accuracy: 0.8611


In [ ]:
label_model.save('label_model')

INFO:tensorflow:Assets written to: label_model/assets


INFO:tensorflow:Assets written to: label_model/assets


In [ ]:
label_model = tf.keras.models.load_model('label_model')

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

def prepare_data(input_text, tokenizer):
    token = tokenizer.encode_plus(
        input_text,
        max_length=50, 
        truncation=True, 
        padding='max_length', 
        add_special_tokens=True,
        return_tensors='tf'
    )
    return {
        'input_ids': tf.cast(token.input_ids, tf.float64),
        'attention_mask': tf.cast(token.attention_mask, tf.float64)
    }

def make_prediction(model, processed_data, classes=['normal', 'Calcified Granuloma',
                            'Calcinosis',  'Opacity', 'Cardiomegaly', 'Spondylosis',
                            'Granulomatous Disease', 'Osteophyte', 'Fractures, Bone']):
    probs = model.predict(processed_data)[0]
    return classes[np.argmax(probs)]

In [ ]:
input_text = input('Enter a finding: ')
processed_data = prepare_data(input_text, tokenizer)
result = make_prediction(label_model, processed_data=processed_data)
print(f"Predicted label  : {result}")

Enter a finding: The cardiac contours are normal. The lungs are clear. Thoracic spondylosis
Predicted label  : normal
